In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
compare_file_name = "../test/compare.csv"

In [3]:
compare_df = pd.read_csv(compare_file_name)

In [4]:
compare_df

,id,class_pred,before,after_x,after_y
0,4_0,DATE,21 February 2017,twenty-one February two thousand and seventeen,the twenty first of february twenty seventeen
1,14_1,DATE,4 March 2017,four March two thousand and seventeen,the fourth of march twenty seventeen
2,16_1,DATE,"March 28, 2017",March twenty-eight two thousand and seventeen,march twenty eighth twenty seventeen
3,20_1,DATE,"April 18, 2017",April eighteen two thousand and seventeen,april eighteenth twenty seventeen
4,32_1,DATE,15 August 2016,fifteen August two thousand and sixteen,the fifteenth of august twenty sixteen
5,45_0,DATE,"February 15, 2017",February fifteen two thousand and seventeen,february fifteenth twenty seventeen
6,59_10,MEASURE,76.2/sq mi,76.2/sq mi,seventy six point two per square miles
7,65_1,DATE,6 December 2016,six December two thousand and sixteen,the sixth of december twenty sixteen
8,67_1,DATE,"February 14, 2017",February fourteen two thousand and seventeen,february fourteenth twenty seventeen
9,70_6,DATE,3 June 1712,"three June one thousand, seven hundred and twelve",the third of june seventeen twelve


In [5]:
compare_df.shape

(6888, 5)

## check list

In [6]:
compare_df["class_pred"].value_counts()

DATE          2370
PLAIN         1087
ELECTRONIC     781
MEASURE        780
LETTERS        624
CARDINAL       447
TELEPHONE      254
DIGIT          224
MONEY          120
DECIMAL         71
VERBATIM        41
FRACTION        37
ADDRESS         26
TIME            21
ORDINAL          3
PUNCT            2
Name: class_pred, dtype: int64

In [7]:
def has_measure_string(token):
    token = str(token)
    mea_strs = ['years', 'year', 'months', 'month', 'weeks', 'week', 'days', 'day',
                'hours', 'minutes', 'min', 'seconds', 'nautical', 'acres', 'hectares',
                'tonnes', 'tons', 'kilobytes', 'nanobarns', 'square', 'miles', 'kilometres',
                'kilometre', 'kilometers', 'kilometer', 'meters', 'meter', 'metres', 'metre',
                'feet', 'yards', 'barrels', 'calories', 'watts', 'knots', 'inches', 'pounds',
                'degrees', 'bar', 'Cellos']
    for one in mea_strs:
        if one in token:
            return True
    return False

DATE 不用再查，表现很好。compare_df[compare_df["class_pred"] == "DATE"]

In [8]:
letter_df = compare_df[compare_df["class_pred"] == "LETTERS"]
letter_df[:50]

,id,class_pred,before,after_x,after_y
29,153_3,LETTERS,IEEE,IEEE,i e e e
38,197_11,LETTERS,IQTREE,IQTREE,i q t r e e
46,224_6,LETTERS,MDCCXLI,MDCCXLI,m d c c x l i
57,296_0,LETTERS,Ubs,Ubs,u b's
64,312_6,LETTERS,IEEE,IEEE,i e e e
69,341_0,LETTERS,LSSMN,LSSMN,l s s m n
87,465_7,LETTERS,CMCL,CMCL,c m c l
92,498_14,LETTERS,'em,'em,e m
93,498_17,LETTERS,'em,'em,e m
97,544_2,LETTERS,P.E.G.,P.E.G.,p e g


letters 暂过

In [7]:
plain_df = compare_df[compare_df["class_pred"] == "PLAIN"]
# plain_df.loc[plain_df['before'] == "-"]
plain_df.shape

(1115, 5)

In [8]:
plain_df_2 = plain_df.loc[plain_df["before"].apply(lambda x: x not in [":", "-"])]
plain_df_2.shape

(53, 5)

In [9]:
plain_df_2

,id,class_pred,before,after_x,after_y
166,1008_7,PLAIN,~,tilde,to
192,1263_1,PLAIN,Dc,d c,Dc
226,1505_10,PLAIN,Hbf,h b f,Hbf
312,2086_1,PLAIN,ZnI₂,ZnI₂,zinc iodide
334,2287_8,PLAIN,mrd,m r d,mrd
433,3040_16,PLAIN,~,tilde,to
541,3929_3,PLAIN,AM3,three a m,AM3
552,3980_1,PLAIN,Glk,g l k,Glk
608,4439_11,PLAIN,NaN,NaN,NaN
645,4713_6,PLAIN,NH₄,NH₄,ammonium


plain 暂过

In [10]:
mea_df = compare_df[compare_df["class_pred"] == "MEASURE"]

In [13]:
mea_df[50: 100]

,id,class_pred,before,after_x,after_y
816,5943_9,MEASURE,32 years,thirty-two years,thirty two years
823,6023_14,MEASURE,"47,000 square",forty-seven thousand square,forty seven thousand square
826,6039_12,MEASURE,124 feet,one hundred and twenty-four feet,one hundred twenty four feet
840,6137_17,MEASURE,"16,991 ft","sixteen thousand, nine hundred and ninety-one ft",sixteen thousand nine hundred ninety one feet
877,6402_19,MEASURE,260 kJ/m³,two hundred and sixty kJ/m³,two hundred sixty kilo joules per cubic meters
888,6504_11,MEASURE,137.7 square miles,137.7 square miles,one hundred thirty seven point seven square miles
893,6541_16,MEASURE,49ers,49ers,forty nine ers
898,6575_10,MEASURE,"6,020.9/km²",6020.9/km²,six thousand twenty point nine per square kilo...
904,6597_11,MEASURE,"1,600 hours","one thousand, six hundred hours",one thousand six hundred hours
920,6756_4,MEASURE,250 years,two hundred and fifty years,two hundred fifty years


MEASURE 初看不错，暂过

In [14]:
compare_df[compare_df["class_pred"] == "ADDRESS"]

,id,class_pred,before,after_x,after_y
322,2146_7,ADDRESS,C000575,C000575,c o o o five seven five
487,3380_7,ADDRESS,B000889,B000889,b o o o eight eight nine
733,5445_4,ADDRESS,I01,I01,i one
891,6514_7,ADDRESS,C000895,C000895,c o o o eight nine five
1013,7444_6,ADDRESS,B000196,B000196,b o o o one nine six
1580,12200_6,ADDRESS,M000016,M000016,m o o o o one six
1637,12776_7,ADDRESS,M000585,M000585,m o o o five eight five
1745,13638_5,ADDRESS,I-247,I-247,i two forty seven
1833,14506_7,ADDRESS,B000336,B000336,b o o o three three six
2695,22601_7,ADDRESS,C000006,C000006,c o o o o o six


In [22]:
card_df = compare_df[compare_df["class_pred"] == "CARDINAL"]
card_df.shape

(664, 5)

In [26]:
card_df[150:200]

,id,class_pred,before,after_x,after_y
2162,17586_12,CARDINAL,"65,178","sixty-five thousand, one hundred and seventy-e...",sixty five thousand one hundred seventy eight
2171,17642_10,CARDINAL,4471,four four seven one,four thousand four hundred seventy one
2177,17662_7,CARDINAL,1961,nineteen sixty one,one thousand nine hundred sixty one
2182,17706_3,CARDINAL,1824,eighteen twenty four,one thousand eight hundred twenty four
2206,18012_5,CARDINAL,LXVIII,l x v i i i,sixty eight
2217,18152_1,CARDINAL,9780299228934,"nine trillion, seven hundred and eighty billio...",nine trillion seven hundred eighty billion two...
2236,18288_2,CARDINAL,"175,615","one hundred and seventy-five thousand, six hun...",one hundred seventy five thousand six hundred ...
2244,18410_3,CARDINAL,220 194,two hundred and twenty one hundred and ninety-...,two hundred twenty thousand one hundred ninety...
2281,18877_2,CARDINAL,5186357,"five million, one hundred and eighty-six thous...",five million one hundred eighty six thousand t...
2305,19123_1,CARDINAL,9780385354042,"nine trillion, seven hundred and eighty billio...",nine trillion seven hundred eighty billion thr...


In [16]:
cv = compare_df[compare_df["class_pred"] == "CARDINAL"]["before"].values
count = 0
for one in cv:
    if one.isdigit() and int(one)<=3000 and int(one) >=1000:
        count += 1
print("count: ", count)

count:  221


In [18]:
compare_df[compare_df["class_pred"] == "ELECTRONIC"]

,id,class_pred,before,after_x,after_y
45,220_11,ELECTRONIC,and/or,and/or,a n d slash o r
69,335_3,ELECTRONIC,#ZumaMustFall,#ZumaMustFall,hash tag zumamustfall
83,428_4,ELECTRONIC,.USA,u s a,dot u s a
101,546_1,ELECTRONIC,Stone.com,Stone.com,s t o n e dot c o m
109,595_0,ELECTRONIC,Investorguide.comReardon,Investorguide.comReardon,i n v e s t o r g u i d e dot c o m r e a r d o n
115,645_2,ELECTRONIC,2016.in,2016.in,t w o z e r o o n e s i x dot i n
135,786_24,ELECTRONIC,http://wwwusers.di.uniroma1.it/~velardi/IEEE_C...,http://wwwusers.di.uniroma1.it/~velardi/IEEE_C...,h t t p colon slash slash w w w u s e r s dot ...
144,828_0,ELECTRONIC,OnlineBeccaloni,OnlineBeccaloni,o n l i n e b e c c a l o n i
156,938_2,ELECTRONIC,http://www.amnh.org/exhibitions/permanent-exhi...,http://www.amnh.org/exhibitions/permanent-exhi...,h t t p colon slash slash w w w dot a m n h do...
164,1001_3,ELECTRONIC,http://www.governor.nsw.gov.au/governor/vice-r...,http://www.governor.nsw.gov.au/governor/vice-r...,h t t p colon slash slash w w w dot g o v e r ...


In [19]:
compare_df[compare_df["class_pred"] == "MONEY"]

,id,class_pred,before,after_x,after_y
54,282_7,MONEY,"US$610,408",US$610408,six hundred ten thousand four hundred eight do...
55,282_9,MONEY,2016 dollars,two thousand sixteen dollars,two thousand sixteen dolla rupees
229,1512_12,MONEY,"US$182,250",US$182250,one hundred eighty two thousand two hundred fi...
230,1512_14,MONEY,"US$42,750",US$42750,forty two thousand seven hundred fifty dollars
231,1512_17,MONEY,"US$37,500",US$37500,thirty seven thousand five hundred dollars
259,1725_9,MONEY,$23.2 billion,$23.2 billion,twenty three point two billion dollars
317,2129_5,MONEY,$ 16.8 billion,$ 16.8 billion,sixteen point eight billion dollars
340,2317_8,MONEY,$1695,$1695,one thousand six hundred ninety five dollars
435,3052_14,MONEY,$4billion,$4billion,fourbillion dollars
454,3201_7,MONEY,$2.00,two dollars,two point o o dollars


In [20]:
compare_df[compare_df["class_pred"] == "ORDINAL"]

,id,class_pred,before,after_x,after_y
125,705_23,ORDINAL,IX,IX,the ninth
353,2460_10,ORDINAL,II,two,the second
430,3016_1,ORDINAL,VI,VI,the sixth
507,3557_2,ORDINAL,VII,seven,the seventh
854,6237_10,ORDINAL,IV,i v,the fourth
906,6603_7,ORDINAL,I,I,the first
930,6805_15,ORDINAL,II,two,the second
1014,7451_7,ORDINAL,I,I,the first
1154,8764_2,ORDINAL,IV,i v,the fourth
1195,9160_1,ORDINAL,41ST,forty first,41ST


In [11]:
compare_df[compare_df["class_pred"] == "TELEPHONE"]

,id,class_pred,before,after_x,after_y
39,413_13,TELEPHONE,1941-1946 (1953) 348-68,one nine four one sil one nine four six sil on...,one nine four one sil one nine four six sil on...
57,587_11,TELEPHONE,045007,forty-five thousand and seven,o four five o o seven
60,624_15,TELEPHONE,330-2600 BCE,330-2600 BCE,three three o sil two six o o sil b c e
61,624_21,TELEPHONE,2600-1900 BCE,2600-1900 BCE,two six o o sil one nine o o sil b c e
93,1039_1,TELEPHONE,2010-2015 SE,2010-2015 SE,two o one o sil two o one five sil s e
130,1522_1,TELEPHONE,978-1-86395-429-7,978-1-86395-429-7,nine seven eight sil one sil eight six three n...
137,1606_10,TELEPHONE,1881-1899 ASIN,one eight eight one sil one eight nine nine si...,one eight eight one sil one eight nine nine si...
155,1926_3,TELEPHONE,978-0-3743-5095-6,978-0-3743-5095-6,nine seven eight sil o sil three seven four th...
172,2110_1,TELEPHONE,4-8179-2022,4-8179-2022,four sil eight one seven nine sil two o two two
173,2115_1,TELEPHONE,22-25 RUS,22-25 RUS,two two sil two five sil r u s
